# Assignment 3

`"Divination is turning out to be much more trouble than I could have foreseen, never having studied the subject myself."
-J.K. Rowling, Harry Potter and the Half-Blood Prince`

---

## Libraries

#### IMPORTANT NOTES:

- Use the `display` function from `IPython.display` for printing pandas dataframes.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
from IPython.display import display
from tsa_functions import *

register_matplotlib_converters()
sns.set_style('darkgrid')

np.set_printoptions(precision=4)
pd.set_option('precision', 4)

import statsmodels.api as sm
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
from statsmodels.tsa.arima.model import ARIMA

In [2]:
import warnings
warnings.filterwarnings("ignore")

from itertools import product
from IPython.display import clear_output
import tqdm

pd.set_option('display.max_colwidth', -1)

sm.__version__

'0.12.2'

In [3]:
def mae(y_true, y_pred):
    score = np.mean(np.abs(y_true - y_pred))
    return score


def rmse(y_true, y_pred):
    if len(y_true) != len(y_pred):
        raise ValueError('Lengths Mismatch')
    score = np.sqrt(np.mean((y_true - y_pred)**2))
    return score


def mase(y_true, y_pred, s_ts):
    if len(y_true) != len(y_pred):
        raise ValueError('Lengths Mismatch')
    ts = s_ts.to_numpy()
    score = np.mean(
        np.abs((y_true - y_pred)/np.mean(np.abs(ts[1:] - ts[:-1]))))
    return score


def rmsse(y_true, y_pred, ts):
    if len(y_true) != len(y_pred):
        raise ValueError('Lengths Mismatch')
    ts = ts.to_numpy()
    score = np.sqrt(
        np.mean(((y_true - y_pred)/np.mean(np.abs(ts[1:] - ts[:-1])))**2))
    return score


def rateMyForecast(train, test, forecast):
    """
    Evalute the forcast per group, given train, test, and forecast tables.

    The function evaluates the metrics per column of the provided table.

    Parameters
    ----------
    train : DataFrame
        DataFrame contaning the train set.
    test : DataFrame
        DataFrame contaning the train set.
    forecast : DataFrame
        DataFrame contaning the train set.

    Returns
    -------
    DataFrame
        DataFrame contaning the metrics as columns, groups as rows,
        and scores as values.

    """
    res = pd.DataFrame([
        {'Group': col,
         'RMSE': rmse(test[col], forecast[col]),
         'MAE': mae(test[col], forecast[col]),
         'MASE': mase(test[col], forecast[col], train[col]),
         'RMSSE': rmsse(test[col], forecast[col], train[col])}
        for col in test])
    display(res.set_index('Group'))
    return res.set_index('Group')


def timeSeriesSplit(design_set, h, val_size):
    val_end = len(design_set)
    divider = val_end - h
    dataset = []
    while len(design_set) - divider <= val_size:
        dataset.append(
            (design_set[np.arange(0, divider)],
             design_set[np.arange(divider, val_end)])
        )
        val_end -= 1
        divider -= 1
    return dataset[::-1]


def scorer(name, est, config, design_set, h, val_size):
    model_params = config.copy()
    fit_params = {}
#     if name == 'ETSModel':
#         fit_params['optimized'] = model_params.pop('optimized'),
#         fit_params['use_boxcox'] = model_params.pop('use_boxcox'),
#         fit_params['remove_bias'] = model_params.pop('remove_bias')

    dataset = timeSeriesSplit(design_set, h, val_size)
    scores = []
    fit = np.nan
    for X, y in dataset:
        try:
            model = est(X, **model_params)
            fit = model.fit(**fit_params)
            y_hat = fit.forecast(h)
            scores.append(rmse(y, y_hat))
        except:
            scores.append(np.nan)

    res = {
        'Model': name,
        'Config': config,
        'Average RMSE': np.nanmean(scores),
        'Stdev RMSE': np.nanstd(scores)}
    res['Sum'] = res['Average RMSE'] + res['Stdev RMSE']
    return res


def bestModel(design_set):

    ########################
    ## Parameter Building ##
    ########################

    seasonal_periods = 4
    h = 8
    val_size = 16

    ets_param_grid = {
        'error': ['add', 'mul'],
        'trend': ['add', 'mul', None],
        'damped_trend': [True, False],
        'seasonal': ['add', 'mul', None],
        'seasonal_periods': [seasonal_periods],
        'initialization_method': ['estimated', 'heuristic', 'known'],
        #         'optimized': [True],
        #         'use_boxcox': [True, False],
        #         'remove_bias': [True, False]

    }
    arm_param_grid = {
        'order': list(product([0, 1, 2, 4], range(0, 3), range(0, 3))),
        'trend': ['n', 'c', 't', 'ct'],
    }

    ets_param_dict = {params: dict(zip(ets_param_grid.keys(), params))
                      for params
                      in product(*ets_param_grid.values())}
    arm_param_dict = {params: dict(zip(arm_param_grid.keys(), params))
                      for params
                      in product(*arm_param_grid.values())}

    ####################
    ## Scoring Proper ##
    ####################

    scores = []
    for name, config in ets_param_dict.items():
        scores.append(
            scorer('ETSModel', ETSModel, config, design_set, h, val_size))
    for name, config in arm_param_dict.items():
        scores.append(
            scorer('ARIMA', ARIMA, config, design_set, h, val_size))

    return pd.DataFrame(scores)


def forecastUsingConfig(regions, design_set, test_set):
    forecast = {}
    for region in regions:
        if region['Model'] == 'ARIMA':
            model = ARIMA(design_set[region['Region']], **region['Config'])
        else:
            model = ETSModel(design_set[region['Region']], **region['Config'])
        fit = model.fit()
        forecast[region['Region']] = fit.forecast(len(df_test))
    forecast_set = pd.DataFrame(forecast)
    forecast_set.index = test_set.index
    return forecast_set

---

## Forecasting Australian Domestic Tourism - REDUX

For this homework, we'll be continuing our exploration of forecasting hierarchical and grouped time series.

Load the Australian domestic tourism dataset and perform the following:

- Recode the `State` variable, similar to what we did in the discussion notebook.

- Create a hierarchical time series with overnight trips in regions at the bottom level of the hierarchy (which can then be aggregated to states, then aggregated to the national `Total`), similar to what we did in the discussion notebook.

- Re-index the series using `pd.period_range` instead of the strings in `Quarter`.

- Withhold the last two years (8 quarters) as a test set. 

- Withhold the last four years (16 quarters) in the training set as a validation set.

In [4]:
state_dict = {
    'New South Wales': 'NSW',
    'Northern Territory': 'NT',
    'Queensland': 'QLD',
    'South Australia': 'SA',
    'Tasmania': 'TAS',
    'Victoria': 'VIC',
    'Western Australia': 'WA'}
df = pd.read_csv('special_datasets/tourism.csv')
df.State.replace(state_dict, inplace=True)

df.index = pd.DatetimeIndex(df.Quarter).to_period('Q')
# df['Year'] = pd.to_datetime(df.Quarter).dt.year
# df['Quarter'] = pd.to_datetime(df.Quarter).dt.quarter

df_pivot = df.pivot_table(
    values='Trips',
    index=df.index,
    columns=['State', 'Region'],
    aggfunc=np.sum)

df_design = df_pivot.loc[: '2015Q4']
df_train = df_pivot.loc[: '2013Q4']
df_val = df_pivot.loc['2012Q1': '2015Q4']
df_test = df_pivot.loc['2016Q1':]
display(df_train, df_val, df_test)

State         ACT            NSW                                            \
Region   Canberra Blue Mountains Capital Country Central Coast Central NSW   
Quarter                                                                      
1998Q1   551.0019  195.5432       261.8081        455.1668      424.3954     
1998Q2   416.0256  200.4087       238.6000        343.4768      557.0116     
1998Q3   436.0290  253.3597       184.4238        303.6709      555.5280     
1998Q4   449.7984  245.5738       229.0889        331.8171      590.1584     
1999Q1   378.5728  290.4841       252.2820        466.8156      480.1293     
...           ...       ...            ...             ...           ...     
2012Q4   511.7661  131.0548       287.3246        340.7241      505.6835     
2013Q1   524.5536  193.2221       304.0950        377.2229      363.1995     
2013Q2   475.5325  181.7916       205.4075        346.6879      560.3985     
2013Q3   506.5121  197.8199       248.7887        258.9110      404.9852     
2013Q4   529.5845  161.3945       301.8307        319.0460      441.1772     

State                                                                 \
Region     Hunter New England North West North Coast NSW Outback NSW   
Quarter                                                                
1998Q1   674.9785  329.8385               1342.3012       96.6027      
1998Q2   791.8872  328.3660               1161.1086       120.6484     
1998Q3   566.0406  324.8592               839.3681        110.0738     
1998Q4   591.5689  279.7962               1183.8600       185.4700     
1999Q1   785.1028  327.4919               1306.8775       78.3638      
...           ...       ...                    ...             ...     
2012Q4   795.4073  350.3062               1199.5305       62.8140      
2013Q1   711.2954  314.9118               1234.2089       45.9733      
2013Q2   680.0313  268.2175               936.4144        104.1240     
2013Q3   697.9204  299.0346               1011.6567       106.6543     
2013Q4   659.9071  288.0075               1114.8221       72.2101      

State              ...            VIC                          \
Region   Riverina  ... Phillip Island Spa Country Upper Yarra   
Quarter            ...                                          
1998Q1   215.2689  ...  364.0433       86.1960     102.7910     
1998Q2   336.0949  ...  166.2436       74.3569     74.8551      
1998Q3   228.0654  ...  83.0328        60.7785     59.4654      
1998Q4   201.1235  ...  215.6779       46.0133     35.2389      
1999Q1   200.8224  ...  263.7101       65.2992     67.8235      
...           ...  ...      ...            ...         ...      
2012Q4   191.4725  ...  169.6084       55.4732     72.0509      
2013Q1   182.3122  ...  246.1275       39.6718     71.5825      
2013Q2   271.8371  ...  188.9554       68.3296     48.1736      
2013Q3   205.5485  ...  117.3503       66.1651     22.0000      
2013Q4   195.8645  ...  185.3366       72.4729     31.6343      

State                                                   WA  \
Region  Western Grampians  Wimmera Australia's Coral Coast   
Quarter                                                      
1998Q1   86.9966           18.8047  132.5164                 
1998Q2   84.9400           52.4823  172.6154                 
1998Q3   79.9749           35.6576  173.9043                 
1998Q4   116.2356          27.2045  207.0026                 
1999Q1   101.7656          50.2199  198.8566                 
...          ...               ...       ...                 
2012Q4   89.8130           43.1522  152.1080                 
2013Q1   59.2859           24.5524  171.1355                 
2013Q2   74.2560           36.4962  168.1334                 
2013Q3   58.3112           15.1660  180.1881                 
2013Q4   101.5272          48.0526  132.3641                 

State                                                      \
Region  Australia's Golden Outback Australia's North West

State         ACT            NSW                                            \
Region   Canberra Blue Mountains Capital Country Central Coast Central NSW   
Quarter                                                                      
2012Q1   446.9046  184.0520       246.4256        359.6829      423.2840     
2012Q2   456.8020  259.4219       294.3512        248.5392      526.6296     
2012Q3   521.8367  190.1276       216.0869        309.3167      394.8585     
2012Q4   511.7661  131.0548       287.3246        340.7241      505.6835     
2013Q1   524.5536  193.2221       304.0950        377.2229      363.1995     
2013Q2   475.5325  181.7916       205.4075        346.6879      560.3985     
2013Q3   506.5121  197.8199       248.7887        258.9110      404.9852     
2013Q4   529.5845  161.3945       301.8307        319.0460      441.1772     
2014Q1   540.6075  179.7695       267.6407        414.3682      339.3733     
2014Q2   586.8275  264.5403       242.8830        296.1988      472.0700     
2014Q3   606.2121  190.8150       236.8738        216.5052      500.9570     
2014Q4   526.9391  202.2309       252.6499        312.2192      535.2692     
2015Q1   566.1355  155.0849       256.9625        418.7622      378.2065     
2015Q2   516.8703  312.2449       215.3308        244.0471      530.0010     
2015Q3   688.2032  191.6364       287.8541        258.2806      473.7973     
2015Q4   597.2456  226.2409       345.6945        339.2346      549.6517     

State                                                                 \
Region     Hunter New England North West North Coast NSW Outback NSW   
Quarter                                                                
2012Q1   703.6991  337.4748               1140.2143       38.9960      
2012Q2   601.2917  329.1314               923.7900        80.2753      
2012Q3   656.9761  340.8776               971.2369        71.9047      
2012Q4   795.4073  350.3062               1199.5305       62.8140      
2013Q1   711.2954  314.9118               1234.2089       45.9733      
2013Q2   680.0313  268.2175               936.4144        104.1240     
2013Q3   697.9204  299.0346               1011.6567       106.6543     
2013Q4   659.9071  288.0075               1114.8221       72.2101      
2014Q1   801.2537  344.7888               1301.4334       47.1369      
2014Q2   692.4411  291.2233               1174.7376       99.9929      
2014Q3   686.8498  310.2272               1021.4664       147.4189     
2014Q4   785.3634  337.7434               1178.5194       104.2203     
2015Q1   838.8067  335.5004               1306.6054       68.1654      
2015Q2   764.0549  379.5845               1193.6271       112.0811     
2015Q3   699.6875  294.5415               1097.9386       126.2728     
2015Q4   831.2333  300.3492               1266.5966       105.1184     

State              ...            VIC                          \
Region   Riverina  ... Phillip Island Spa Country Upper Yarra   
Quarter            ...                                          
2012Q1   183.6027  ...  315.5053       78.7224     60.2400      
2012Q2   217.9396  ...  147.4012       79.3044     68.1378      
2012Q3   176.5125  ...  126.0946       57.1883     48.5398      
2012Q4   191.4725  ...  169.6084       55.4732     72.0509      
2013Q1   182.3122  ...  246.1275       39.6718     71.5825      
2013Q2   271.8371  ...  188.9554       68.3296     48.1736      
2013Q3   205.5485  ...  117.3503       66.1651     22.0000      
2013Q4   195.8645  ...  185.3366       72.4729     31.6343      
2014Q1   217.1528  ...  326.7374       108.1304    71.0182      
2014Q2   217.3002  ...  164.1070       71.9461     45.4003      
2014Q3   236.5091  ...  96.3691        66.5930     29.0583      
2014Q4   254.2813  ...  203.0495       63.2429     76.8980      
2015Q1   253.1515  ...  266.8429       88.3428     48.0880      
2015Q2   260.6241  ...  194.8407       71.6348     65.1461      
2015Q3   264.2596  ...  123.6403       87.3

State         ACT            NSW                                            \
Region   Canberra Blue Mountains Capital Country Central Coast Central NSW   
Quarter                                                                      
2016Q1   625.1416  227.4607       306.4441        398.0859      508.6850     
2016Q2   592.6085  303.5584       322.5047        250.9560      461.0432     
2016Q3   572.4371  211.0181       262.9282        265.4521      505.2368     
2016Q4   667.2141  220.1716       277.3330        379.4839      525.2546     
2017Q1   634.3687  190.4739       300.0212        455.0586      529.9431     
2017Q2   748.2904  315.7889       305.7111        278.4784      651.3597     
2017Q3   631.7599  314.6307       287.2512        272.4384      628.5964     
2017Q4   720.3294  203.4606       356.3841        427.7658      628.2281     

State                                                                  \
Region      Hunter New England North West North Coast NSW Outback NSW   
Quarter                                                                 
2016Q1   842.8019   321.2152               1560.8200       65.6794      
2016Q2   829.9771   392.0172               1153.6977       134.8313     
2016Q3   801.0572   350.3716               1038.7536       141.2427     
2016Q4   843.7873   346.6872               1407.1824       105.9467     
2017Q1   757.0282   358.0866               1409.7407       91.1370      
2017Q2   897.9597   405.9132               1164.0613       107.5434     
2017Q3   914.8204   398.9737               1145.6544       162.5924     
2017Q4   1004.3339  404.1934               1329.1106       106.3099     

State              ...            VIC                          \
Region   Riverina  ... Phillip Island Spa Country Upper Yarra   
Quarter            ...                                          
2016Q1   238.9226  ...  327.2870       91.9398     38.0968      
2016Q2   279.9532  ...  168.6530       82.1689     63.6975      
2016Q3   254.5432  ...  145.2257       77.7021     52.0486      
2016Q4   230.6681  ...  189.9613       114.6199    59.6499      
2017Q1   182.6422  ...  339.8957       87.9224     114.1359     
2017Q2   293.5650  ...  196.4758       56.1299     93.9915      
2017Q3   213.9023  ...  156.0623       100.1157    39.8043      
2017Q4   218.1398  ...  238.1329       95.3488     75.0986      

State                                                   WA  \
Region  Western Grampians  Wimmera Australia's Coral Coast   
Quarter                                                      
2016Q1   95.9714           42.6252  284.5467                 
2016Q2   49.5275           24.4022  279.9112                 
2016Q3   70.2469           24.4363  254.0475                 
2016Q4   102.3744          25.7099  256.8122                 
2017Q1   92.9616           62.9338  241.9544                 
2017Q2   71.5968           41.8446  223.8558                 
2017Q3   56.8456           62.5967  282.3629                 
2017Q4   119.9106          45.1500  274.7947                 

State                                                      \
Region  Australia's Golden Outback Australia's North West   
Quarter                                                     
2016Q1   278.3108                   209.7269                
2016Q2   288.9768                   289.8143                
2016Q3   271.1387                   424.9869                
2016Q4   269.9291                   316.2068                
2017Q1   225.9662                   226.5482                
2017Q2   256.3540                   285.7901                
2017Q3   306.1539                   352.1124                
2017Q4   223.7437                   288.6869                

State                                            
Region  Australia's South West Experience Perth  
Quarter                                          
2016Q1   982.0733               1116.5143        
2016Q2   650.1197               960.5722         
2016Q3   531.237

### Q1.

Given that:

1. Our goal is to create an 8-step forecasting model.

2. We have allocated 16 observations for the validation set. 

How many train-val splits will we end up with under TSCV?

In [5]:
a = np.zeros((9, 16))
for i, arr in enumerate(a):
    arr[0:i+8] = 1
    arr[:i] = -1
df_0 = pd.DataFrame(a.astype(int))
df_0.columns = range(-16, 0)
df_0.index = range(1, 10)
df_0.style.background_gradient(cmap='coolwarm', axis=None)

,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1
1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0
2,-1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0
3,-1,-1,1,1,1,1,1,1,1,1,0,0,0,0,0,0
4,-1,-1,-1,1,1,1,1,1,1,1,1,0,0,0,0,0
5,-1,-1,-1,-1,1,1,1,1,1,1,1,1,0,0,0,0
6,-1,-1,-1,-1,-1,1,1,1,1,1,1,1,1,0,0,0
7,-1,-1,-1,-1,-1,-1,1,1,1,1,1,1,1,1,0,0
8,-1,-1,-1,-1,-1,-1,-1,1,1,1,1,1,1,1,1,0
9,-1,-1,-1,-1,-1,-1,-1,-1,1,1,1,1,1,1,1,1


### Answer.

`9 train-val splits`

### Q2.

Extract the `Canberra` time series.

Read the `statsmodels` API entries on [`ETS`](https://www.statsmodels.org/stable/generated/statsmodels.tsa.exponential_smoothing.ets.ETSModel.html) and [`ARIMA`](https://www.statsmodels.org/stable/generated/statsmodels.tsa.arima.model.ARIMA.html) to determine the different hyperparameters/configurations that you can select. 

Recall that `ARIMA` models have certain [assumptions](https://otexts.com/fpp3/stationarity.html).

Using TSCV on the validation set, find the best performing `ETS` and `ARIMA` model based on the sum of their average and stdev. `RMSE` values.

Compile the `RMSE` statistics of both models in a dataframe and print it (i.e. avg, stdev, avg + stdev).

In [6]:
# Your code here
res = bestModel(df_design[('ACT', 'Canberra')])

In [8]:
(res
 .groupby('Model', as_index=False)
 .apply(lambda x: x.iloc[x.Sum.argmin()])
)

,Model,Config,Average RMSE,Stdev RMSE,Sum
0,ARIMA,"{'order': (1, 0, 0), 'trend': 't'}",44.0876,12.9451,57.0327
1,ETSModel,"{'error': 'add', 'trend': 'mul', 'damped_trend': False, 'seasonal': None, 'seasonal_periods': 4, 'initialization_method': 'heuristic'}",55.0465,12.3809,67.4274


### Q3.

Choose the best model from the two above. Evaluate it on the `Canberra` test set.

Compile the `MAE`, `RMSE`, `MASE`, and `RMSSE` metrics in a dataframe and print it.

In [9]:
regions = [{
    'Region': ('ACT', 'Canberra') ,
    **res.loc[res['Sum'].idxmin()].to_dict()}]
df_forecast = forecastUsingConfig(regions, df_design, df_test)

In [10]:
rateMyForecast(
    df_design[[('ACT', 'Canberra')]],
    df_test[[('ACT', 'Canberra')]],
    df_forecast[[('ACT', 'Canberra')]]);

,RMSE,MAE,MASE,RMSSE
Group,,,,
"(ACT, Canberra)",51.3659,39.9172,0.6221,0.8006


### Q4.

Repeat the process done in Q2 and Q3 <b><u>for all regions</b></u>.

Only print out the final dataframe. Sample shown below.

In [12]:
regions = []
for label, content in tqdm.tqdm(list(df_design.items())):
    regions.append({
        'Region': label,
        **bestModel(content).nsmallest(1, 'Sum').iloc[0].to_dict()
    })

100%|██████████| 76/76 [2:51:39<00:00, 135.53s/it]  


In [13]:
pd.DataFrame(regions)

,Region,Model,Config,Average RMSE,Stdev RMSE,Sum
0,"(ACT, Canberra)",ARIMA,"{'order': (1, 0, 0), 'trend': 't'}",44.0876,12.9451,57.0327
1,"(NSW, Blue Mountains)",ETSModel,"{'error': 'mul', 'trend': 'add', 'damped_trend': False, 'seasonal': 'add', 'seasonal_periods': 4, 'initialization_method': 'heuristic'}",34.5277,7.9519,42.4797
2,"(NSW, Capital Country)",ARIMA,"{'order': (1, 0, 2), 'trend': 't'}",33.1587,4.6608,37.8195
3,"(NSW, Central Coast)",ARIMA,"{'order': (2, 0, 2), 'trend': 'c'}",33.8040,3.6350,37.4390
4,"(NSW, Central NSW)",ETSModel,"{'error': 'mul', 'trend': None, 'damped_trend': False, 'seasonal': 'add', 'seasonal_periods': 4, 'initialization_method': 'estimated'}",56.8041,5.5173,62.3214
...,...,...,...,...,...,...
71,"(WA, Australia's Coral Coast)",ARIMA,"{'order': (0, 0, 2), 'trend': 't'}",64.7244,15.7177,80.4421
72,"(WA, Australia's Golden Outback)",ARIMA,"{'order': (0, 0, 2), 'trend': 't'}",64.0100,9.6084,73.6184
73,"(WA, Australia's North West)",ETSModel,"{'error': 'mul', 'trend': 'mul', 'damped_trend': False, 'seasonal': 'add', 'seasonal_periods': 4, 'initialization_method': 'heuristic'}",55.2003,18.4021,73.6024
74,"(WA, Australia's South West)",ARIMA,"{'order': (4, 0, 2), 'trend': 't'}",166.2667,55.0986,221.3654


In [19]:
df_forecast = forecastUsingConfig(regions, df_design, df_test)

In [54]:
rms_region = rateMyForecast(
    df_design,
    df_test,
    df_forecast)
clear_output()
rms_region['Best Model'] = [d['Model'] + str(d['Config']) for d in regions]
rms_region.sort_index(axis=1)

,Best Model,MAE,MASE,RMSE,RMSSE
Group,,,,,
"(ACT, Canberra)","ARIMA{'order': (1, 0, 0), 'trend': 't'}",39.9172,0.6221,51.3659,0.8006
"(NSW, Blue Mountains)","ETSModel{'error': 'mul', 'trend': 'add', 'damped_trend': False, 'seasonal': 'add', 'seasonal_periods': 4, 'initialization_method': 'heuristic'}",30.8866,0.8423,35.6142,0.9712
"(NSW, Capital Country)","ARIMA{'order': (1, 0, 2), 'trend': 't'}",34.3025,0.7922,42.7229,0.9867
"(NSW, Central Coast)","ARIMA{'order': (2, 0, 2), 'trend': 'c'}",35.4584,0.4389,43.4205,0.5375
"(NSW, Central NSW)","ETSModel{'error': 'mul', 'trend': None, 'damped_trend': False, 'seasonal': 'add', 'seasonal_periods': 4, 'initialization_method': 'estimated'}",83.4478,1.3060,96.6310,1.5123
...,...,...,...,...,...
"(WA, Australia's Coral Coast)","ARIMA{'order': (0, 0, 2), 'trend': 't'}",37.0062,1.1588,41.6100,1.3030
"(WA, Australia's Golden Outback)","ARIMA{'order': (0, 0, 2), 'trend': 't'}",32.6179,1.1410,40.5184,1.4173
"(WA, Australia's North West)","ETSModel{'error': 'mul', 'trend': 'mul', 'damped_trend': False, 'seasonal': 'add', 'seasonal_periods': 4, 'initialization_method': 'heuristic'}",216.3964,4.1747,235.4822,4.5429


### Q5.

Produce aggregate forecasts for the `State` and `Total` levels using the regional forecasts (i.e. bottom-up).

Calculate the average `MAE`, `RMSE`, `MASE`, and `RMSSE` across all series (i.e. `Region`, `State`, `Total`).

In [63]:
display(df_forecast.sum(level=0, axis=1))
rms_state = rateMyForecast(
    df_train.sum(level=0, axis=1),
    df_test.sum(level=0, axis=1),
    df_forecast.sum(level=0, axis=1))

,ACT,NSW,NT,QLD,SA,TAS,VIC,WA
Quarter,,,,,,,,
2016Q1,604.2125,7826.4100,224.1544,5004.8305,1685.8357,934.0161,5971.5984,2783.8184
2016Q2,611.2139,7327.2933,442.4822,5491.3730,1536.7123,678.3240,5190.6624,2897.3751
2016Q3,618.2485,7069.0324,601.6263,5834.4816,1431.5213,491.8589,4750.7931,2724.1374
2016Q4,625.3151,7600.4262,344.2640,5426.4269,1598.0540,693.7848,5351.9794,2919.2845
2017Q1,632.4125,7973.6466,227.3203,5167.6455,1720.8535,921.6617,5996.4570,3110.6390
2017Q2,639.5397,7447.2461,447.1472,5672.7010,1564.9640,672.0829,5243.7651,3092.1543
2017Q3,646.6956,7191.4208,608.0696,6021.3368,1458.6066,492.5985,4791.8063,3062.7930
2017Q4,653.8791,7728.9161,348.4789,5600.0549,1625.4481,686.3289,5383.2495,3149.8696


,RMSE,MAE,MASE,RMSSE
Group,,,,
ACT,51.3659,39.9172,0.6211,0.7992
NSW,637.9279,569.0431,1.2632,1.4161
NT,90.2720,66.0411,0.4155,0.5679
QLD,329.2314,238.9838,0.6201,0.8542
SA,133.3043,117.3410,0.7238,0.8223
TAS,126.4874,114.8961,0.5063,0.5574
VIC,902.7367,805.4750,1.2387,1.3883
WA,465.3150,432.7160,4.7080,5.0626


In [61]:
df_forecast_total = df_forecast.sum(axis=1).rename('Total').to_frame()
display(df_forecast_total)
rms_total = rateMyForecast(
    df_train.sum(axis=1).rename('Total').to_frame(),
    df_test.sum(axis=1).rename('Total').to_frame(),
    df_forecast_total)

,Total
Quarter,
2016Q1,25034.8761
2016Q2,24175.4361
2016Q3,23521.6996
2016Q4,24559.5347
2017Q1,25750.6360
2017Q2,24779.6002
2017Q3,24273.3272
2017Q4,25176.2250


,RMSE,MAE,MASE,RMSSE
Group,,,,
Total,1656.7145,1490.3895,1.3834,1.5378


<div class="alert alert-block alert-info">
Calculating the average for all 85 (1 Total + 8 States + 76 Regions) time series forecasts as done in the book. (https://otexts.com/fpp3/tourism.html)
</div>


In [76]:
(pd
 .concat([rms_region, rms_state, rms_total])
 .mean()
 .rename('All Series')
 .to_frame()
 .T)

,RMSE,MAE,MASE,RMSSE
All Series,97.2011,83.5789,0.8997,1.0778
